<a href="https://colab.research.google.com/github/mohamad-tohidi/NLP/blob/main/OCR%2BNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing the necessery parts

In [ ]:
!pip install -q tesseract pytesseract
!sudo apt install tesseract-ocr

In [ ]:
!pip install clean-text numpy hazm emoji

In [ ]:
!pip install -qU sentence-transformers

In [ ]:
"""keep in mind after installing you have to restart your session
بعد از نصب پکیج های جدید سشن کولب رو ریستارت کنید"""

In [1]:
!wget -q https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/examples/applications/text-summarization/LexRank.py

Copying OCR model to PATH

In [2]:
#/usr/share/tesseract-ocr/4.00/tessdata/

import shutil

shutil.copyfile("/content/drive/MyDrive/BestOcrEver/fas.traineddata","/usr/share/tesseract-ocr/4.00/tessdata/fas.traineddata")

'/usr/share/tesseract-ocr/4.00/tessdata/fas.traineddata'

##Importing Libraries

In [3]:
%cd /content/drive/MyDrive/BestOcrEver/Nevise

/content/drive/MyDrive/BestOcrEver/Nevise


SpellChecker imports

In [4]:
import os
from main import Normalizer
from main import load_pre_model
from main import spell_check

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

tesseracts

In [5]:
import cv2
import pytesseract
from PIL import Image

Summery

In [6]:
import numpy as np
import pandas as pd

import hazm
import requests
import time

import torch
from sentence_transformers import models, SentenceTransformer, util
from LexRank import degree_centrality_scores

Semmantic search

In [7]:
from sentence_transformers import models, SentenceTransformer, util

from io import open


##Defining SpellChecker model

SpellCheker

In [ ]:
normalizer = Normalizer()
vocab_path = os.path.join('model', 'vocab.pkl')
model_checkpoint_path = os.path.join('model', 'model.pth.tar')
model, vocab, device = load_pre_model(vocab_path=vocab_path, model_checkpoint_path=model_checkpoint_path)
    #test

##Functions

OCR

In [9]:
def perform_ocr_on_image(image_path):
    # Load the image using OpenCV
    image = cv2.imread(image_path)

    # Perform OCR using PyTesseract
    ocr_text = pytesseract.image_to_string(image, lang = 'fas')

    # # Save the OCR results to a text file
    # with open("ocr_results.txt", "w") as ocr_file:
    #     ocr_file.write(ocr_text)
    return ocr_text



SpellCheck

In [10]:

def ocr_and_spell_check(image_path):
    # Perform OCR on the image
    extracted_text = pytesseract.image_to_string(Image.open(image_path), lang = 'fas')

    # Remove unwanted characters
    cleaned_text = extracted_text.replace('\n', ' ').replace('\u200c', '').replace('\x0c', '')

    # Spell check the text
    corrected_text = spell_check(model, vocab, device, normalizer, cleaned_text)

    # Save the original and corrected texts to files
    original_file_path = "OCR_text.txt"
    corrected_file_path = "corrected_OCR_text.txt"

    with open(original_file_path, "w", encoding="utf-8") as original_file:
        original_file.write(extracted_text)

    with open(corrected_file_path, "w", encoding="utf-8") as corrected_file:
        corrected_file.write(corrected_text)

    print(f"Original text saved to {original_file_path}")
    print(f"Corrected text saved to {corrected_file_path}")

Summery

In [11]:
def load_st_model(model_name_or_path):
    word_embedding_model = models.Transformer(model_name_or_path)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False)

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

In [12]:
def summarize_text(input_filename, output_filename, model_name='m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens'):
    # Load the Sentence-Transformer model
    embedder = SentenceTransformer(model_name)

    # Read content from the input file
    with open(input_filename, 'r', encoding='utf-8') as file:
        content = file.read()

    # Split the document into sentences
    sentences = hazm.sent_tokenize(content)

    # Compute the sentence embeddings
    embeddings = embedder.encode(sentences, convert_to_tensor=True, show_progress_bar=True)

    # Compute the pair-wise cosine similarities
    cos_scores = util.pytorch_cos_sim(embeddings, embeddings).numpy()

    # Compute centrality scores for each sentence
    centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

    # Sort sentences by centrality scores
    most_central_sentence_indices = np.argsort(-centrality_scores)

    # Extract the top 4 sentences as the summary
    summary_sentences = [sentences[idx].strip() for idx in most_central_sentence_indices[:4]]

    # Save the summarized content to a new file
    with open(output_filename, 'w', encoding='utf-8') as output_file:
        output_file.write('\n'.join(summary_sentences))


Sentiment search

In [13]:
# Function to read a text file and turn each sentence into a corpus list
def load_corpus(file_path):
    """
    Reads a text file and returns a list of sentences and a list of sentences encoded using the Sentence-Transformer model.

    Args:
      file_path: The path to the text file.

    Returns:
      corpus: A list of sentences.
      corpus_embeddings: A list of sentence embeddings.
    """
    corpus = []
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            corpus.append(line)

    # Load the Sentence-Transformer model
    embedder = SentenceTransformer('m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens')

    # Encode the corpus sentences
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True, show_progress_bar=True)

    return corpus, corpus_embeddings


In [14]:
# Function to perform semantic search on a corpus
def semantic_search(corpus, corpus_embeddings):
    """
    Performs semantic search on a corpus of sentences.

    Args:
      corpus: A list of sentences.
      corpus_embeddings: A list of sentence embeddings.

    Returns:
      None
    """
    # Get user input for the query
    query = input("Enter your query: ")

    embedder = SentenceTransformer('m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens')

    # Encode the query sentence
    query_embedding = embedder.encode(query, convert_to_tensor=True, show_progress_bar=True)

    # Calculate the cosine similarity between the query and the corpus sentences
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    # Find the top 5 most similar sentences
    top_k = 1
    top_results = torch.topk(cos_scores, k=top_k)

    # Print the results
    print(f"Query: {query}")
    for score, idx in zip(top_results[0], top_results[1]):
        print(f"{corpus[idx]} --- (Score: {score:.4f})")

##Q&A

dummy test

In [ ]:
from transformers import pipeline

model_name = "SajjadAyoubi/bert-base-fa-qa"
qa_pipeline = pipeline("question-answering", model=model_name, tokenizer=model_name)

text = """وفقیت چاپ اول کتاب اقتصاد خرد میانه باعث خوشحالی من شد. این حقیقت باعث شد که عقیدة من مبنی بر اینکه بازار یک روش تحلیلی را برای اقتصاد خرد در سطح کارشناسی پذیرفته می‌کند. تاکید شود. هدف من از نوشتن چاپ اول این کتاب ارائة طرز عمل روشهای اقتصاد خرد بود تا به دانشجویان امکان دهد که از این ابزار بهروش خودشان,و نه بهصورت انتخاب موارد از پیش حلشده در متن کتاب,استفاده کنند."""
questions = ["از این ابزار به چه شکلی استفاده میکنن؟", "اسم کتاب چیه؟", "هدف چی بود؟"]

for question in questions:
    print(qa_pipeline({"context": text, "question": question}))

real func

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import pytesseract
from PIL import Image
from main import spell_check

def ocr_and_spell_check(image_path, corrected_text_path):
    # Perform OCR on the image
    extracted_text = pytesseract.image_to_string(Image.open(image_path), lang='fas')

    # Remove unwanted characters
    cleaned_text = extracted_text.replace('\n', ' ').replace('\u200c', '').replace('\x0c', '')

    # Spell check the text
    corrected_text = spell_check(model, vocab, device, normalizer, cleaned_text)

    # Save the corrected text to a file
    with open(corrected_text_path, 'w', encoding='utf-8') as corrected_file:
        corrected_file.write(corrected_text)

    print(f"Corrected text saved to {corrected_text_path}")

    return corrected_text

def answer_questions(model_name_or_path, context):
    # Load the question-answering model
    nlp_qa = pipeline('question-answering', model=model_name_or_path, tokenizer=model_name_or_path)

    # Ask user for questions
    questions_to_ask = []
    num_questions = int(input("Enter the number of questions you want to ask: "))

    for i in range(num_questions):
        question = input(f"Enter question {i+1}: ")
        questions_to_ask.append(question)

    # Ask and answer questions
    answers = []
    for question in questions_to_ask:
        result = nlp_qa(question=question, context=context)
        answer = result["answer"].strip()
        answers.append(answer)
        print(f"Q: {question}\nA: {answer}\n")

    # Save questions and answers to a TXT file
    qa_output_path = 'qa_output.txt'
    with open(qa_output_path, 'w', encoding='utf-8') as qa_file:
        for question, answer in zip(questions_to_ask, answers):
            qa_file.write(f"Q: {question}\nA: {answer}\n\n")

    print(f"Questions and answers saved to {qa_output_path}")

# Example usage:
# image_path = '/content/ph.jpg'
# corrected_text_path = 'corrected_text.txt'

# # OCR and Spell Check
# corrected_text = ocr_and_spell_check(image_path, corrected_text_path)

# # Question Answering
# answer_questions("m3hrdadfi/xlmr-large-qa-fa", corrected_text)

##PipeLine

In [ ]:
""" First we use ocr_and_spell_check to perform ocr and SpellChek it
it will save two txt files in path
then we use Summerize function to save a summerized version of the Corrected txt file"""

image_path = "/content/drive/MyDrive/BestOcrEver/test.jpg"
ocr_and_spell_check(image_path)

In [ ]:
input_file = "corrected_OCR_text.txt"
output_file = "Summerized.txt"
summarize_text(input_file, output_file)

In [ ]:
correcte_text = "path to correctede text"
answer_questions("m3hrdadfi/xlmr-large-qa-fa", corrected_text)


Semantic search

In [ ]:
file_path = "/content/drive/MyDrive/BestOcrEver/Nevise/corrected_OCR_text.txt"
corpus, corpus_embeddings = load_corpus(file_path)

# Perform semantic search
semantic_search(corpus, corpus_embeddings)